In [1]:
import geopandas as gpd
import pandas as pd
import copy
import os
import warnings
import random
from pprint import pprint
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from srai.datasets import AirbnbMulticityDataset
from srai.embedders import Hex2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.plotting import plot_regions
from srai.regionalizers import H3Regionalizer
import plotly.graph_objs as go
from shapely.geometry import LineString, Point
from shapely import from_geojson
import h3
from srai.h3 import h3_to_geoseries
import matplotlib.pyplot as plt

In [2]:
H3_RESOLUTION = 9
seq_length = 8

In [3]:
embeddings = pd.read_parquet('embeddings_9.parquet')
embeddings.shape

(221407, 10)

In [4]:
gdf = gpd.read_parquet('../geolife_points_smooth.parquet')
gdf.shape

(149173, 12)

# GDF AGGREAGATION

In [5]:
gdf_agg = gdf.groupby('trajectory_id').agg(
    {'geometry': LineString, 'date_str': list, 'speed': list, 'direction': list, 'altitude': list,
     'trajectory_id': 'first'})
gdf_agg.shape

(1795, 6)

In [6]:
gdf_agg.head()

,geometry,date_str,speed,direction,altitude,trajectory_id
trajectory_id,,,,,,
20070412101853_0,"LINESTRING (116.33028 39.97552, 116.33057 39.9...","[2007-04-12, 2007-04-12]","[0.3174578150582134, 0.3174578150582134]","[15.013957914870332, 20.964606804911114]","[351.049868766404, 118.110236220472]",20070412101853_0
20070412102325_0,"LINESTRING (116.33040 39.97585, 116.33057 39.9...","[2007-04-12, 2007-04-12]","[0.24933972268054067, 0.24933972268054067]","[20.964676832270413, 20.964606804911114]","[114.829396325459, 118.110236220472]",20070412102325_0
20070413105648_0,"LINESTRING (116.33015 39.97622, 116.33028 39.9...","[2007-04-13, 2007-04-13, 2007-04-13, 2007-04-1...","[0.07796726674907895, 0.07796726674907895, 0.0...","[99.26439871977664, 99.26439871977664, 213.973...","[0.0, 55.7742782152231, 55.7742782152231, 170....",20070413105648_0
20070413150314_0,"LINESTRING (116.31905 39.92687, 116.31551 39.9...","[2007-04-13, 2007-04-13, 2007-04-13, 2007-04-13]","[9.889579092536993, 9.889579092536993, 0.48658...","[269.68467918410283, 269.68467918410283, 336.9...","[252.624671916011, 252.624671916011, 262.46719...",20070413150314_0
20070414022829_0,"LINESTRING (116.31554 39.92693, 116.31654 39.9...","[2007-04-14, 2007-04-14, 2007-04-14, 2007-04-1...","[1.2964931291294477, 1.2964931291294477, 1.443...","[94.8087951122991, 104.61717890145906, 64.6302...","[242.782152230971, 193.569553805774, 147.63779...",20070414022829_0


In [7]:
# gdf_agg = gdf_agg[:1000].copy()

In [8]:
def get_hex_neigh(traj):
    h3_dist = []
    h3_cells = []
    points = traj.coords
    for idx in range(len(points) - 1):
        a, b = points[idx], points[idx + 1]
        start_hex = h3.latlng_to_cell(a[1], a[0], H3_RESOLUTION)
        end_hex = h3.latlng_to_cell(b[1], b[0], H3_RESOLUTION)
        if start_hex == end_hex:
            continue
        for h3_cell in h3.grid_path_cells(start_hex, end_hex):
            if not h3_cells or h3_dist[-1] != h3_cell:
                # print(h3.grid_disk(h3_cell, 1))
                h3_dist.append(h3_cell)
                h3_cells.append(h3.grid_disk(h3_cell, 1))
    return h3_cells

In [9]:
dummies = pd.get_dummies([0, 1, 2, 3, 4, 5, 6])

In [10]:
def get_y(traj):
    if not traj:
        return None
    y = []
    for i in range(len(traj) - 1):
        target = traj[i + 1][0]
        prop = np.array(traj[i])
        res = prop == target
        # print(res)
        idx = np.where(res)[0]
        try:
            dummy = dummies.loc[idx].values[0]
        except:
            print(traj)
        y.append(dummy)
    y.append(dummies.loc[0].values)
    return y

In [11]:
ls = gdf_agg['geometry'].iloc[5]
h3_cells = []
points = ls.coords
for idx in range(len(points) - 1):
    a, b = points[idx], points[idx + 1]
    start_hex = h3.latlng_to_cell(a[1], a[0], H3_RESOLUTION)
    end_hex = h3.latlng_to_cell(b[1], b[0], H3_RESOLUTION)
    if start_hex == end_hex:
        continue
    for h3_cell in h3.grid_path_cells(start_hex, end_hex):
        if not h3_cells or h3_cells[-1] != h3_cell:
            h3_cells.append(h3_cell)
m = h3_to_geoseries(h3_cells).reset_index().explore("index", tiles="CartoDB positron", opacity=0.4)
gpd.GeoSeries([ls]).explore(m=m)

In [12]:
t = gdf_agg['geometry'].iloc[5]
cells = get_hex_neigh(t)
pprint(len(cells))
y = get_y(cells)
print(len(y))
# em = embed(cells)
# pprint(em[0])

28
28


In [13]:
def embed(traj):
    emb = []
    for t in traj:
        temp = []
        for h in t:
            try:
                temp.append(embeddings.loc[h].values)
            except:
                return None
        emb.append(temp)
    return emb

In [14]:
gdf_agg['hex_neigh'] = gdf_agg['geometry'].apply(get_hex_neigh)
gdf_agg['y'] = gdf_agg['hex_neigh'].apply(get_y)
gdf_agg['neigh_emb'] = gdf_agg['hex_neigh'].apply(embed)

In [15]:
gdf_agg = gdf_agg[gdf_agg['neigh_emb'].notnull()]
gdf_agg = gdf_agg[gdf_agg['y'].notnull()]
gdf_agg = gdf_agg[gdf_agg['y'].apply(lambda x: len(x) >= seq_length)]

In [16]:
len(gdf_agg.iloc[0, 0].coords), len(gdf_agg.iloc[0, -1])

(23, 28)

In [17]:
len(gdf_agg.iloc[2, -2:]['y']), len(gdf_agg.iloc[2, -2:]['neigh_emb'])

(19, 19)

# SEQUENCE GENERATION

In [18]:
def make_seq(row, seq_len=seq_length):
    # print(len(row['neigh_emb']), len(row['y']))
    for i in range(0, len(row['y']) - seq_len):
        # yield np.array(row['neigh_emb'][i:i+seq_len]).flatten().tolist(), row['y'][i+seq_len]
        # yield row['neigh_emb'][i:i+seq_len], row['y'][i+seq_len]
        seq = []
        for j in range(i, i+seq_len):
            seq.append(np.array(row['neigh_emb'][j]).flatten().tolist())
        yield seq, row['y'][i+seq_len]
            

In [ ]:
gdf_seq = gpd.GeoDataFrame(list(row for _, row in gdf_agg.iterrows() for row in make_seq(row)) , columns=['seq', 'y'])

In [ ]:
gdf_seq.head() 

In [ ]:
seq_expanded = pd.DataFrame(gdf_seq['seq'].tolist(), index=gdf_seq.index)

In [ ]:
seq_expanded.head()

In [ ]:
X = seq_expanded
y = gdf_seq['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Saving data to train

In [ ]:
data_path = 'input_data'
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [ ]:
path = f'data_res{H3_RESOLUTION}_seq{seq_length}'
path = os.path.join(data_path, path)
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
torch.save((X_train, y_train), os.path.join(path, 'train.pt'))
torch.save((X_test, y_test), os.path.join(path, 'test.pt'))